In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from google.cloud.storage.blob import Blob
from google.cloud import storage

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Method to write dataframe to the GCP bucket
'''
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mlb-mvp-predictions-571fb58bbad3.json'

client = storage.Client()
bucket = client.get_bucket('football-hackathan')
bucket.blob('corr_matrix.csv').upload_from_string(corr_matrix.to_csv(), 'text/csv')
'''

"\nos.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mlb-mvp-predictions-571fb58bbad3.json'\n\nclient = storage.Client()\nbucket = client.get_bucket('football-hackathan')\nbucket.blob('corr_matrix.csv').upload_from_string(corr_matrix.to_csv(), 'text/csv')\n"

In [3]:
soccer_file = pd.read_csv('gs://football-hackathan/train.csv')
soccer_test = pd.read_csv('gs://football-hackathan/test.csv')

# Data Processing

### Handling the missing data

In [9]:
# number of rows with missing entries
soccer_file.isnull().any(axis = 1).sum()

19566

In [10]:
# number of null values by column
missing_list = list(soccer_file.isnull().sum())
col_list = list(soccer_file.columns)
missing_percent = []
for val in missing_list:
    percentage = val/len(soccer_file)
    missing_percent.append(percentage)


missing_dict = tuple(zip(col_list, missing_list, missing_percent))
missing_df = pd.DataFrame(missing_dict, columns = ['column_title', 'abs_missing', 'pct_missing'])
    


In [11]:
large_missing = missing_df.sort_values(by = ['abs_missing'], ascending = False)
large_missing

,column_title,abs_missing,pct_missing
392,player_physical_derived_var_131,7409,0.362245
418,player_other_derived_var_157,7409,0.362245
355,player_physical_derived_var_94,7409,0.362245
289,player_physical_derived_var_28,7409,0.362245
320,player_physical_derived_var_59,7409,0.362245
...,...,...,...
12,player_general_var_2,0,0.000000
13,player_general_var_3,0,0.000000
15,player_general_var_5,0,0.000000
1,scout_id,0,0.000000


In [12]:
# columns with more than 40% missing values
df_missing = missing_df[missing_df['pct_missing'] >= 0.4]
missing_cols = list(df_missing['column_title'])

In [13]:
soccer_file = soccer_file.drop(columns = missing_cols)
soccer_test = soccer_test.drop(columns = missing_cols)

In [14]:
# One-hot encode the categorical variables in the dataset
soccer_file = pd.get_dummies(soccer_file, columns = ['winner', 'team', 'player_position_1', 'player_position_2'])
soccer_test = pd.get_dummies(soccer_test, columns = ['winner', 'team', 'player_position_1', 'player_position_2'])

In [15]:
# fill the missing values of the dataframe with the mean values
soccer_file = soccer_file.apply(lambda x: x.fillna(x.mean()), axis = 0)

In [16]:
soccer_test = soccer_test.apply(lambda x: x.fillna(x.mean()), axis = 0)
soccer_test = soccer_test.drop(columns = 'row_id')

In [17]:
# number of rows with missing entries
soccer_test.isnull().any(axis = 1).sum()

0

In [18]:
# Isolate the rating_num dependent variable from the rest of the features
rating_dep = soccer_file['rating_num']
ind_vars = soccer_file.drop(columns = ['rating_num', 'row_id'])

In [19]:
ind_vars.shape

(20453, 774)

In [20]:
# Normalize the independent variables on a scale of 0 to 1
scaler = MinMaxScaler()

scale = MinMaxScaler().fit(ind_vars)
soccer_train = scale.transform(ind_vars)

# apply the same normalization to the testing dataset
soccer_test_norm = scale.transform(soccer_test)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- player_position_1_21.0
- player_position_2_21.0
Feature names seen at fit time, yet now missing:
- player_position_1_22.0
- player_position_2_22.0

  warnings.warn(message, FutureWarning)


In [21]:
soccer_train = pd.DataFrame(soccer_train, columns = ind_vars.columns)
# soccer_train['rating_num'] = rating_dep

soccer_test_norm = pd.DataFrame(soccer_test_norm, columns = soccer_test.columns)

# Random Forest feature selection

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SelectFromModel

In [23]:
x = soccer_train
y = rating_dep

In [24]:
# split the training dataset into a further training/testing split
# allows for evaluation of the model performance
# x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, shuffle = True, random_state = 1337)

In [28]:
random_forest = RandomForestRegressor(n_estimators = 150)
random_forest.fit(x, y)

RandomForestRegressor(n_estimators=150)

In [29]:
# create a feature importance plot
importance = random_forest.feature_importances_.argsort()

In [30]:
# plt.barh(soccer_train.columns[importance], random_forest.feature_importances_[importance])
# plt.xlabel('feature importance')

In [31]:
cols = soccer_train.columns[importance]
importances = random_forest.feature_importances_[importance]

In [32]:
importance_df = pd.DataFrame(list(zip(cols, importances)), columns = ['feature', 'importance'])

In [33]:
order_importance = importance_df.sort_values(by = ['importance'], ascending = False)
order_importance

,feature,importance
773,player_positional_raw_var_185,0.035981
772,winner_loser,0.021939
771,scout_id,0.015484
770,competitionId,0.012584
769,player_offensive_derived_var_55,0.009505
...,...,...
2,player_position_2_22.0,0.000000
1,player_offensive_raw_var_156,0.000000
4,player_position_1_22.0,0.000000
5,player_position_1_20.0,0.000000


# Linear Regression - Random Forest selected variables

In [128]:
# select the variables with the x highest rated importance
# order_importance = pd.read_csv('program_output/random_forest_importance.csv')
# order_importance = order_importance.drop(columns = ['Unnamed: 0'])

In [41]:
order_importance

,feature,importance
773,player_positional_raw_var_185,0.035981
772,winner_loser,0.021939
771,scout_id,0.015484
770,competitionId,0.012584
769,player_offensive_derived_var_55,0.009505
...,...,...
2,player_position_2_22.0,0.000000
1,player_offensive_raw_var_156,0.000000
4,player_position_1_22.0,0.000000
5,player_position_1_20.0,0.000000


In [42]:
features = list(order_importance['feature'])[0:50]

In [43]:
features

['player_positional_raw_var_185',
 'winner_loser',
 'scout_id',
 'competitionId',
 'player_offensive_derived_var_55',
 'player_other_raw_var_38',
 'player_offensive_derived_var_89',
 'player_offensive_derived_var_127',
 'player_weight',
 'player_height',
 'player_other_raw_var_197',
 'player_other_derived_var_42',
 'player_general_var_4',
 'player_positional_derived_var_85',
 'player_positional_ratio_var_46',
 'team2_other_ratio_var_15',
 'player_other_derived_var_76',
 'player_other_raw_var_109',
 'player_other_ratio_var_11',
 'team2_other_raw_var_36',
 'player_other_ratio_var_61',
 'team2_other_raw_var_49',
 'player_offensive_ratio_var_45',
 'player_general_derived_var_146',
 'player_offensive_derived_var_24',
 'player_offensive_derived_var_90',
 'player_positional_derived_var_92',
 'team2_offensive_derived_var_19',
 'player_offensive_derived_var_154',
 'team1_other_raw_var_36',
 'player_other_derived_var_145',
 'team1_other_raw_var_49',
 'team1_other_ratio_var_15',
 'player_defensiv

In [44]:
forest_frame = soccer_train[features]
forest_frame['rating_num'] = rating_dep
forest_frame_test = soccer_test_norm[features]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [45]:
forest_frame = forest_frame.drop(columns = ['scout_id'])
forest_frame_test = forest_frame_test.drop(columns = ['scout_id'])

In [48]:
forest_frame_test

,player_positional_raw_var_185,winner_loser,competitionId,player_offensive_derived_var_55,player_other_raw_var_38,player_offensive_derived_var_89,player_offensive_derived_var_127,player_weight,player_height,player_other_raw_var_197,...,player_other_ratio_var_43,player_physical_raw_var_196,player_offensive_derived_var_151,team1_other_ratio_var_22,player_other_ratio_var_9,player_positional_derived_var_130,player_other_ratio_var_41,team1_other_ratio_var_16,team2_defensive_derived_var_10,team1_other_ratio_var_35
0,0.309368,0.0,0.272727,0.220294,0.000000,0.316975,0.308869,0.416667,0.307692,0.539088,...,0.373,0.248373,0.501639,0.421687,0.429,0.079032,0.627,0.336538,0.279219,0.388800
1,0.067538,0.0,0.454545,0.145358,0.000000,0.164734,0.181439,0.583333,0.564103,0.539088,...,0.250,0.248373,0.420397,0.353414,0.154,0.039640,0.750,0.371795,0.360020,0.449600
2,0.254902,1.0,0.545455,0.102518,0.000000,0.236755,0.241741,0.270833,0.179487,0.539088,...,0.447,0.248373,0.431282,0.445783,0.400,0.091178,0.553,0.461538,0.364730,0.572800
3,0.163399,1.0,0.636364,0.171319,0.066667,0.235680,0.265362,0.270833,0.333333,0.492188,...,0.333,0.167539,0.306720,0.510040,0.375,0.078778,0.667,0.387821,0.515167,0.364800
4,0.100218,0.0,0.090909,0.284642,0.000000,0.400990,0.398251,0.687500,0.589744,0.625000,...,0.583,0.183246,0.247264,0.405622,0.571,0.037099,0.417,0.352564,0.411699,0.571200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8769,0.095861,1.0,0.272727,0.120829,0.133333,0.107128,0.114332,0.583333,0.461538,0.742188,...,0.333,0.146597,0.348271,0.329317,0.250,0.046275,0.667,0.471154,0.386332,0.342400
8770,0.296296,0.0,0.000000,0.334367,0.200000,0.376092,0.385732,0.375000,0.333333,0.539088,...,0.317,0.248373,0.475270,0.273092,0.692,0.096745,0.683,0.346154,0.395247,0.649600
8771,0.063181,1.0,0.000000,0.265464,0.000000,0.217921,0.235031,0.333333,0.435897,0.820313,...,0.526,0.364747,0.342295,0.244980,0.500,0.041401,0.474,0.391026,0.153401,0.548800
8772,0.259259,0.0,0.545455,0.319748,0.000000,0.408740,0.421627,0.520833,0.461538,0.562500,...,0.478,0.254799,0.608588,0.415942,0.600,0.115741,0.522,0.383853,0.252675,0.558341


In [49]:
x_train = forest_frame.iloc[:,:-1]
y_train = forest_frame.iloc[:, -1]

In [150]:
# split the training dataset into a further training/testing split
# allows for evaluation of the model performance
# x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.25, shuffle = True, random_state = 1337)

In [51]:
linear_regression = LinearRegression()

In [53]:
# train the linear regression model
linear_regression.fit(x_train, y_train)

LinearRegression()

In [55]:
rating_pred

array([7.24811896, 6.45209816, 6.04505874, ..., 6.75279723, 6.79668627,
       7.47903805])

In [54]:
rating_pred = linear_regression.predict(forest_frame_test)

In [154]:
# mean squared error
# print('mean squared error: %.2f' % mean_squared_error(y_test, rating_pred))
# r2 value
# print('r2 of the linear regression: %.2f' % r2_score(y_test, rating_pred))

mean squared error: 2.91
r2 of the linear regression: 0.14


# CSV file for submission

In [58]:
soccer_test = pd.read_csv('gs://football-hackathan/test.csv')

In [59]:
row_id = list(soccer_test['row_id'])
rating_num = list(rating_pred)

In [61]:
submission_df = pd.DataFrame(list(zip(row_id, rating_num)), columns = ['row_id','rating_num'])

In [64]:
submission_df.to_csv('program_output/lin_regression_submission.csv', index = False)